# Lab3.3 Training an emotion classifier using embeddings

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

### Table of Contents

* [Section 1: Quick introduction to embeddings](#section1)
* [Section 2: Loading the emotion data](#section2)
* [Section 3: Preparing the training and test data](#section3)
* [Section 4: Training and applying the model](#section4)
* [Section 5: Generating the test report](#section5)
* [Section 6: Applying the classifier to your own text](#section6)


## 1 Quick introduction to embeddings  <a class="anchor" id ="section1"></a> 

Extracting features manually can get us a long way. In addition to lemma and part-of-speech, people have used other information: features of the previous words (on the left) or the next words (on the right), whether the current word starts with a capital, whether it is an abbreviation, etc.

A recent alternative way to create a 'semantic' representation of a word is by word embeddings: mapping words (or phrases) from the vocabulary to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with many dimensions per word to a continuous vector space with a much lower dimension. For this reason, they are called dense representations.

In linguistics, word embeddings were discussed in the research area of distributional semantics. The idea is to quantify and categorize semantic similarities between linguistic items based on their distributional properties in large samples of language data. The underlying notion is that "a word is characterized by the company it keeps" (Firth). Embeddings are however the weights in the hidden layer of a neural network that is trained to predict the contexts rather than representing the context in a vector directly.

### Reference:

For a nice explanation how word embedddings can improve classical bag-of-word approaches, check out this page:

https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html


In this section, we will load pre-trained word embeddings called word2vec, created by Google. The embeddings have 300 dimensions.

First, download the file from [Kaggle](https://www.kaggle.com/pkugoodspeed/nlpword2vecembeddingspretrained) or from [Google code archive](https://code.google.com/archive/p/word2vec/). Then, create a folder and unpack the word2vec file in that folder.

We will load the embedding model with the Gensim package that we used before.

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import gensim
import json

In [2]:
##### Change to path to the location of your local copy of the GoogleNews embeddings
##### It may take a  minute to load the model

# path_to_model = '/Users/piek/Desktop/ONDERWIJS/data/word-embeddings/classical-models/GoogleNews-vectors-negative300.bin'
# word_embedding_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  

# ### this model has 300 dimensions so we set the number of features to 300
# num_features = 300

In case you computer cannot handle big models such as the Google news model, you can also download one of the smaller 'Glove' datasets. These are provided as text files and 'gensim' provides a function to convert and load them. Note that these models have less domensions than the Google model and you need to adapt the number of features in the code below.

In [3]:
# from gensim.test.utils import datapath, get_tmpfile
# from gensim.models import KeyedVectors
# from gensim.scripts.glove2word2vec import glove2word2vec

# glove_file = datapath('/Users/piek/Desktop/ONDERWIJS/data/word-embeddings/classical-models/glove.6B.50d.txt')
# tmp_file = get_tmpfile("test_word2vec.txt")

# _ = glove2word2vec(glove_file, tmp_file)
# word_embedding_model = KeyedVectors.load_word2vec_format(tmp_file)

# ### this model has 50 dimensions so we set the number of features to 50
# num_features = 50

Instead of downloading the models to disk, 'gensim' also provides a downloader API to load the model from the web when needed. In the next cell, we use this API to download a word embeddding model trained on tweets. Note that these are GloVe embeddings built using Tweets as the name suggests. These vectors are based on 2B tweets, 27B tokens, 1.2M vocab, uncased. The original source can be found here: https://nlp.stanford.edu/projects/glove/. The 25 in the model name refers to the dimensionality of the vectors.

In [4]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

# Inspect available models
# print(json.dumps(api.info(), indent=4))

# download the model and return as object ready for use
word_embedding_model = api.load("glove-wiki-gigaword-300")

## this model has 25 dimensions so we set the number of features to 25
num_features = 300

Let's check if the model works.

In [5]:
word1='cat'
word2='dog'
word1_vector=np.array(word_embedding_model[word1]).reshape(1, -1)
word2_vector=np.array(word_embedding_model[word2]).reshape(1, -1)
print(cosine_similarity(word1_vector, word2_vector))

[[0.6816747]]


## 2. Loading the emotion data  <a class="anchor" id ="section2"></a> 

In [6]:
import pandas as pd
filepath = './data/MELD/train_sent_emo.csv'
df = pd.read_csv(filepath)

# 3. Preparing the training and test data  <a class="anchor" id ="section3"></a> 

The following import are needed again:

In [7]:
import sklearn
import numpy as np
import nltk
from nltk.corpus import stopwords

In the previous notebook, we used CountVectorizer to obtain the full vocabulary of the data set and generate vectors for the one-hot-endcoing of each word. In these vectors, each slot represents a word and a value '1' indicates that the word was present in the utterance and a '0' means absence. This results is large and sparse vector representations for each utterance. We have also seen that we can weight the relevance of a word using the 'TF.IDF' function. This still results in large and sparse vectors but weights are more subtle. The down side is sparseness, lack of generalisation and lack robustness. 

In the following, we are going to represent the utterances by an embedding representation. In fact, we take the word embedding of each token in the utterance and add these together, after which we take the average. All the embeddings have the same number of dimensions in the same order. So if two tokens have a high weight for one dimension then their co-uccurrence in an utterance will enforce that weight. Note that by adding and taking the average, we normalize for the length of the utterance and the order of the tokens is not relevant.

We are going to define two customized function using 'def' to create an embedding representation for each utterance. These functions are taken from: https://www.kaggle.com/varun08/sentiment-analysis-using-word2vec

The first function, called 'featureVecMethod', takes the words of the utterance and the embedding model as parameters. The num_features parameter determines the size of the vector. 

In [8]:
unknown_words =[]
known_words = []
# Function to average all word vectors in a paragraph
def featureVecMethod(words, stopwords, model, modelword_index, num_features):
    # Pre-initialising empty numpy array for speed
    # This create a numpy array with the length of the num_features set to zero values
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
        
    for word in  words:
        if not word in stop_words: 
            if word in index2word_set:
                nwords = nwords + 1
                featureVec = np.add(featureVec,model[word])
                known_words.append(word)
            else:
                word = word.lower()
                if word in index2word_set:
                    nwords = nwords + 1
                    featureVec = np.add(featureVec,model[word])
                    #we keep track of the words detected
                    known_words.append(word)
                else:
                    #we keep track of the unknown words to see how well our model fits the data
                    unknown_words.append(word)
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

The next function just deals with all the data and creates the list of input vectors. This function calls the previous function

In [9]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(texts, stopwords, model, modelword_index, num_features):
    counter = 0
    textFeatureVecs = np.zeros((len(texts),num_features),dtype="float32")
    for text in texts:
        # Printing a status message every 1000th text
        if counter%200 == 0:
            print("Review %d of %d"%(counter,len(texts)))
            
        textFeatureVecs[counter] = featureVecMethod(text, stopwords, model, modelword_index,num_features)
        counter = counter+1
    return textFeatureVecs

Now back to our input data. We iterate over the Pandas frame in the same way as before but now we extract for each utterance the embedding representation.

In [10]:
# Calculating average feature vector for training set
### This is the number of dimensions in the word2vec model used. 
###The Google news model has 300 dimensions but if you use a Glove model you may have to adapt this accordinlgy

#Converting Index2Word which is a list to a set for better speed in the execution.
#Allows for quicker lookup if the words exist
index2word_set = set(word_embedding_model.wv.index2word)
stop_words = set(stopwords.words('english'))

training_vectors = []
training_labels = []
for index, utterance in enumerate(df['Utterance']):
    ### Running this for all data requires a lot of memory and takes about an hour.
    ### For teaching purposes, it makes sense to limit the data
    ### we limit the data to the first 1000 utterances
    ##if index==2000:
    ##    break
    training_vectors.append(nltk.tokenize.word_tokenize(utterance))
    training_labels.append(df['Emotion'].iloc[index])

trainDataVecs = getAvgFeatureVecs(training_vectors, stop_words, word_embedding_model, index2word_set, num_features)
#### Due to the averaging, there could be infinitive values or NaN values. The next numpy function turns these value to "0" scores
trainDataVecs = np.nan_to_num(trainDataVecs)  

<ipython-input-10-8d4afdae1641>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  index2word_set = set(word_embedding_model.wv.index2word)


Review 0 of 9989
Review 200 of 9989
Review 400 of 9989
Review 600 of 9989
Review 800 of 9989
Review 1000 of 9989
Review 1200 of 9989
Review 1400 of 9989
Review 1600 of 9989
Review 1800 of 9989
Review 2000 of 9989
Review 2200 of 9989
Review 2400 of 9989
Review 2600 of 9989
Review 2800 of 9989
Review 3000 of 9989
Review 3200 of 9989
Review 3400 of 9989
Review 3600 of 9989
Review 3800 of 9989
Review 4000 of 9989
Review 4200 of 9989
Review 4400 of 9989
Review 4600 of 9989
Review 4800 of 9989
Review 5000 of 9989
Review 5200 of 9989
Review 5400 of 9989
Review 5600 of 9989
Review 5800 of 9989
Review 6000 of 9989
Review 6200 of 9989
Review 6400 of 9989
Review 6600 of 9989
Review 6800 of 9989
Review 7000 of 9989
Review 7200 of 9989
Review 7400 of 9989
Review 7600 of 9989
Review 7800 of 9989
Review 8000 of 9989
Review 8200 of 9989
Review 8400 of 9989
Review 8600 of 9989
Review 8800 of 9989
Review 9000 of 9989
Review 9200 of 9989
Review 9400 of 9989
Review 9600 of 9989
Review 9800 of 9989


<ipython-input-8-c153e6840d56>:27: RuntimeWarning: invalid value encountered in true_divide
  featureVec = np.divide(featureVec, nwords)


Training the classifier may take a while. If you laptop cannot handle it, reduce the number of training data. Alternatively, you can use a smaller word2vec embeddings model. Here is a website with many ready to use models: http://vectors.nlpl.eu/repository/

You can either choose a model with a smaller vocabulary or with less dimensions. Whatever you choose, make sure you can load the model using the 'gensim' package. If you choose a model with less than 300 dimensions (e.g. 100 or 200), you also need to adapt the value for *num_features* accordingly = 300.

Let's inspect our training data a bit more. Depending on the break set for loading the training data, you will have a list of vectors with according length:

In [11]:
len(trainDataVecs)

9989

We can inspect the first element in the list:

In [12]:
print('Vector length', len(trainDataVecs[0]))
print (trainDataVecs[0])

Vector length 300
[-1.37924299e-01  1.16590567e-01  5.68268560e-02 -1.67267710e-01
 -5.91286039e-03 -3.27249989e-02  2.77890004e-02  1.09937146e-01
  2.03285292e-01 -1.83727145e+00  5.68199940e-02  1.19454011e-01
 -7.03205690e-02 -1.61910020e-02  1.99671417e-01  6.28664643e-02
 -2.65530020e-01 -1.37555972e-01  5.46851419e-02 -3.20324227e-02
  1.18145846e-01  2.18618840e-01  6.41374523e-03  9.28612947e-02
 -1.49793282e-01 -7.96867162e-02 -6.23320267e-02 -1.17406892e-02
 -7.88672939e-02  1.04428865e-01  1.46555707e-01  3.48418295e-01
 -2.18492702e-01  6.74508512e-02 -7.99954414e-01  2.19417140e-01
 -8.36073514e-03  2.86857132e-02 -6.95371106e-02  6.07981421e-02
 -9.52264294e-02 -1.33950129e-01 -1.26415431e-01  2.10388601e-02
 -6.17462806e-02  1.54575139e-01  2.58392721e-01  2.58471578e-01
 -9.45980027e-02  2.36114319e-02  1.23451455e-02 -8.32298547e-02
  1.98540002e-01 -4.39258553e-02 -5.24611473e-02  9.43685845e-02
  3.01804245e-02  2.20677152e-01  1.29673332e-01  1.06574290e-01
  2.478

It is simply a list with digits, each representing the averaged weight of the tokens or words that made up the utterance. We can checks the length, which should be '300', '100', '50' or '25', etc. depending on the number of dimensions of the word2vec model that you used.

There are two major differences with the bag-of-tokens that we used in the previous notebook:

1. the vectors are short
2. there are no zero's 

Instead of *large sparse* vectors, we now have *short dense* vectors representing each utterance. Whereas in the previous representation, each slot in the vector corresponds with a token, now each slot is a weight from the hidden layer to learn to predict others words in the context.

This is true for each utterance, each having a unique set of values for the same hidden layer weights. These weights now represent the meaning of the utterance for a machine, which can use a similarity function such as cosine similairty to measure the degree of equivalence across these representations. When we inspects any other utterance, we see it is represented in a simlar way.

In [13]:
print(len(trainDataVecs[1000]))
print(trainDataVecs[1000])

300
[-0.16865    -0.071476    0.0975308  -0.2938716   0.082104    0.0152144
 -0.08172165  0.27428198  0.1996912  -1.97246     0.36207598 -0.11237179
 -0.1900876   0.32143396 -0.136188    0.05884483 -0.285274    0.018557
  0.06247123 -0.0881788  -0.0156726   0.324522    0.28636     0.1654242
 -0.24520597 -0.122745    0.0141228  -0.25903136 -0.12345479  0.07944966
 -0.13026461  0.35229802 -0.328014    0.0653716  -1.24339     0.31695202
 -0.02130622  0.20119819 -0.1479394  -0.04315908  0.25983602 -0.22102681
 -0.058835    0.005592   -0.016297    0.0627322   0.38388997  0.38245597
  0.05021999  0.14013237  0.013308   -0.18288441  0.0873314  -0.19051301
 -0.23326962  0.3297742  -0.069718    0.22329001  0.23871557 -0.052416
  0.30157    -0.10474928  0.25386816 -0.058166   -0.13663751 -0.404972
  0.320788   -0.11074319  0.160044   -0.0753178  -0.08160261 -0.12300919
 -0.1053618   0.20837203 -0.1643672  -0.189079    0.06758799  0.192006
 -0.13722448 -0.08483861 -0.081459    0.0034722   0.42817

Since the vectors are compatible, we can compare them in the same way as we did before for the word2vec embeddings of *cat* and *dog*:

In [14]:
word1_vector=np.array(trainDataVecs[0]).reshape(1, -1)
word2_vector=np.array(trainDataVecs[1000]).reshape(1, -1)
print(cosine_similarity(word1_vector, word2_vector))

[[0.8313157]]


For training, we use the same labels as before:

In [15]:
print(training_labels[0], training_labels[1])

neutral neutral


So now we have a numeric representation of each text, based on the embeddings of the words. We feed this to a classifier in the same way as we did in the previous notebooks with the Countvectorizer output.

Before we can train the classifier, we stil need to convert the labels to numeric values as we did before.

Before we do that, it may be good to check which words are not in the embedding model and therefore do not contribute to the representation of the utterance. In the above function, we kept track of the unknown words. Now we can inspect this list. We use the *Counter* function to get a frequency count of these words.

In [16]:
from collections import Counter

unknown_words_count = Counter(unknown_words)
print('Proportion of unknown tokens', len(unknown_words)/(len(unknown_words)+len(known_words)))
print('Number of unknown words',len(unknown_words_count))
print('Number of unknown word tokens:', len(unknown_words))
print('Unknown words counts')
print(unknown_words_count)

Proportion of unknown tokens 0.0650210716435882
Number of unknown words 959
Number of unknown word tokens: 4752
Unknown words counts
Counter({'i\x92m': 496, 'don\x92t': 371, 'it\x92s': 334, 'that\x92s': 248, 'you\x92re': 222, "y'know": 158, 'y\x92know': 157, 'can\x92t': 132, 'we\x92re': 98, 'i\x92ll': 95, 'pheebs': 78, 'he\x92s': 76, 'i\x92ve': 75, 'didn\x92t': 75, 'she\x92s': 59, 'there\x92s': 58, 'let\x92s': 52, 'what\x92s': 49, 'they\x92re': 43, 'doesn\x92t': 40, 'i\x92d': 33, '\x91cause': 31, 'we\x92ll': 28, 'wouldn\x92t': 27, 'and-and': 26, 'you\x92ve': 24, 'you\x92ll': 22, 'you-you': 21, 'won\x92t': 21, 'couldn\x92t': 19, 'who\x92s': 19, 'no-no-no': 18, 'wasn\x92t': 18, 'haven\x92t': 17, '\x91em': 16, 'i\x92m-i\x92m': 16, 'i-i-i': 16, 'isn\x92t': 15, 'here\x92s': 14, 'hey-hey': 14, 'what-what': 13, 'aren\x92t': 12, 'you\x92d': 12, 'no-no-no-no': 11, 'shouldn\x92t': 11, 'we-we': 11, 'doin\x92': 11, 'monica\x92s': 11, 'chandler\x92s': 10, 'oh-oh': 10, 'it\x92s-it\x92s': 9, 'that-th

We also kept track of the *known* words, so lets check these as well:

In [17]:
known_words_count = Counter(known_words)
print('Number of known words',len(known_words_count))
print('Number of known words tokens:', len(known_words))
print('Known words counts')
print(known_words_count)

Number of known words 5064
Number of known words tokens: 68332
Known words counts
Counter({',': 7237, '.': 6509, '!': 4266, 'i': 3422, '?': 2887, 'oh': 1063, 'okay': 665, "'s": 661, 'yeah': 650, 'well': 534, 'you': 513, 'know': 503, 'hey': 497, 'na': 482, 'no': 471, 'right': 466, 'what': 425, "n't": 401, 'go': 378, 'like': 369, 'gon': 366, 'get': 364, '...': 349, 'really': 338, 'got': 328, "'m": 298, 'think': 294, 'uh': 287, 'so': 279, 'look': 275, 'one': 264, 'and': 248, 'mean': 241, 'see': 228, 'come': 220, 'god': 217, 'ross': 213, 'want': 207, 'sorry': 206, 'all': 201, "'re": 195, 'good': 192, 'tell': 190, 'joey': 186, 'great': 175, 'it': 173, 'guys': 169, 'would': 166, 'could': 162, 'hi': 161, 'going': 160, 'that': 158, 'but': 157, 'umm': 156, 'yes': 155, 'time': 149, 'back': 143, 'we': 140, 'little': 139, 'say': 139, 'love': 132, 'something': 130, 'chandler': 130, 'let': 123, 'guy': 120, 'rachel': 119, 'wan': 116, 'monica': 116, 'phoebe': 116, 'thank': 112, 'this': 111, 'why': 110

Just as in the previous notebook, we need to turn the labels into numerical values:

In [18]:
from sklearn import preprocessing
# first we instantiate a label encode
le = preprocessing.LabelEncoder()
# we fee this encoder with the complete list of labels from our data
le.fit(training_labels)
print(list(le.classes_))
training_classes = le.transform(training_labels)
print(list(training_classes[0:20]))

['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
[4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 2, 4, 6, 4, 6, 5, 6, 2, 4, 4]


The next steps are the same as for the previous notebook, except that we pass the embedding representations of the training data.

In [19]:
# Split data into training and test sets
# from sklearn.cross_validation import train_test_split  # deprecated in 0.18
from sklearn.model_selection import train_test_split

### we again use a aplit of 80% train and 20% test
docs_train, docs_test, y_train, y_test = train_test_split(
    trainDataVecs, # the tf-idf model
    training_classes, # the category values for each utterance represented as numeric values
    test_size = 0.20 # we use 80% for training and 20% for development
    ) 


In [20]:
print(y_test)
print(type(y_test))

[4 4 4 ... 3 3 4]
<class 'numpy.ndarray'>


In [21]:
print(docs_train[0:5])

[[-0.01277     0.096773    0.08699    ... -0.2966223  -0.12225801
   0.10452   ]
 [-0.07478025 -0.32193992 -0.06477267 ...  0.06779801 -0.04594325
   0.48933765]
 [-0.15103249  0.03697302 -0.03843162 ... -0.23420173 -0.07220712
   0.161805  ]
 [-0.04567    -0.04644666 -0.01707034 ...  0.09721199 -0.20731331
   0.54734665]
 [-0.08337785  0.13862328 -0.19454452 ... -0.22462443  0.074938
   0.13926242]]


## 4. Training and applying the model  <a class="anchor" id ="section4"></a> 

In [22]:
from sklearn import svm
from sklearn.calibration import CalibratedClassifierCV

# We choose a Linear model
linear_model = svm.LinearSVC()

# The next function is needed to get some confidence score from this model. This score is derived by 10-folded cross-validation: *cv=10* and using a sigmoid function
svm_linear_clf = CalibratedClassifierCV(linear_model , method='sigmoid', cv=10)

### we train the classifier through the *fit* function and by passing the training vectors and the training labels as paramters:
svm_linear_clf.fit(docs_train, y_train)

/usr/local/anaconda3/envs/ma-hlt-labs/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/envs/ma-hlt-labs/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/envs/ma-hlt-labs/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/envs/ma-hlt-labs/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/envs/ma-hlt-labs/lib/python3.8/site-pac

CalibratedClassifierCV(base_estimator=LinearSVC(), cv=10)

In [23]:
# Predicting the Test set results, find macro recall
y_pred_svm_linear = svm_linear_clf.predict(docs_test)

If the data is complex, a non-linear SVM may be preferable. A non-linear SVM uses the kernel-trick to separate positive and negative cases when the data is not lineary correlated. We can initialise such a classifier in the same way as done before.

In [24]:
nonlinear_model = svm.SVC(probability=True)
svm_nonlinear_clf = CalibratedClassifierCV(nonlinear_model,  method='sigmoid', cv=5)
svm_nonlinear_clf.fit(docs_train, y_train)

CalibratedClassifierCV(base_estimator=SVC(probability=True), cv=5)

In [25]:
# Predicting the Test set results, find macro recall
y_pred_svm_nonlinear = svm_nonlinear_clf.predict(docs_test)

In [26]:
# the next function obtain the scores for all the classes for each text
y_pred_svm_nonlinear_proba= svm_nonlinear_clf.predict_proba(docs_test)
print(y_pred_svm_nonlinear_proba)

[[0.05385151 0.01622981 0.03317709 ... 0.55634578 0.03197617 0.24133155]
 [0.08911457 0.02598158 0.02880671 ... 0.59493934 0.07425639 0.01488291]
 [0.05317317 0.01759521 0.03565937 ... 0.60130468 0.10231549 0.02112431]
 ...
 [0.4442592  0.02236026 0.04454602 ... 0.05619502 0.04272649 0.16765925]
 [0.06705054 0.03336897 0.01703803 ... 0.55178175 0.0861462  0.02323939]
 [0.03795664 0.03741348 0.01701748 ... 0.60452065 0.11181592 0.01796443]]


## 5. Generating the test report  <a class="anchor" id ="section5"></a> 

In [27]:
from sklearn.metrics import classification_report


In [28]:
#### this report gives the results for the LINEAR classifier
report = classification_report(y_test,y_pred_svm_linear,digits = 7)
print(le.classes_)
print('SVM LINEAR ----------------------------------------------------------------')
print(report)

['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'surprise']
SVM LINEAR ----------------------------------------------------------------
              precision    recall  f1-score   support

           0  0.3372093 0.1479592 0.2056738       196
           1  0.0000000 0.0000000 0.0000000        44
           2  0.0000000 0.0000000 0.0000000        63
           3  0.5630252 0.3611860 0.4400657       371
           4  0.5822700 0.9302575 0.7162330       932
           5  0.4375000 0.0482759 0.0869565       145
           6  0.5421687 0.3643725 0.4358354       247

    accuracy                      0.5640641      1998
   macro avg  0.3517390 0.2645787 0.2692520      1998
weighted avg  0.5080101 0.5640641 0.4961790      1998



In [29]:
# this report gives the results for the nonlinear classifier
report = classification_report(y_test,y_pred_svm_nonlinear,digits = 7)
print(le.classes_)
print('SVM NONLINEAR ----------------------------------------------------------------')
print(report)

['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'surprise']
SVM NONLINEAR ----------------------------------------------------------------
              precision    recall  f1-score   support

           0  0.3193277 0.1938776 0.2412698       196
           1  0.0000000 0.0000000 0.0000000        44
           2  0.0000000 0.0000000 0.0000000        63
           3  0.5688406 0.4231806 0.4853168       371
           4  0.6068493 0.9506438 0.7408027       932
           5  0.0000000 0.0000000 0.0000000       145
           6  0.6083916 0.3522267 0.4461538       247

    accuracy                      0.5845846      1998
   macro avg  0.3004870 0.2742755 0.2733633      1998
weighted avg  0.4952374 0.5845846 0.5144993      1998



/usr/local/anaconda3/envs/ma-hlt-labs/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Remember the results from the notebook where we trained a NaiveBayes and SVM classifiers with one-hot-encodings of the words? Take some time to compare the results and think about the differences.

## 6. Applying the model to new data  <a class="anchor" id ="section6"></a> 

We would like to apply the embedding based model to our own data but this works a bit different as we cannot simply use the 'transform' function to represent the utterances using the one-hot vector representation of the training vocabulary.

What we need to do is to create an embedding representation using the same function we used above and assume that our classifier finds sufficient similarity in the embeddings of our data with the correct training data.

We use the same set of utterances.

In [30]:
# some utterances
some_chat = ['That is sweet of you', 
               'You are so funny', 
               'Are you a man or a woman?', 
               'Chatbots make me sad and feel lonely.', 
               'Your are stupid and boring.', 
               'Two thumbs up', 
               'I fell asleep halfway through this conversation', 
               'Wow, I am really amazed.', 
               'You are amazing.']


len(some_chat)

9

Next, we define the list of labels that go with our chat.

In [31]:
some_chat_emotions = ['joy', 'joy', 'neutral', 'sadness', 'anger', 'joy', 'anger', 'surprise', 'joy']

We  use the LabelEncoder *le* to convert this list into a numpy array with digits:

In [32]:
print('labels',le.classes_)
some_chat_labels = le.transform(some_chat_emotions)
print(some_chat_labels)

labels ['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'surprise']
[3 3 4 5 0 3 0 6 3]


In [33]:
some_chat_tokens = []
for utterance in some_chat:
    some_chat_tokens.append(nltk.tokenize.word_tokenize(utterance))

some_chat_embedding_vectors = getAvgFeatureVecs(some_chat_tokens, stop_words, word_embedding_model, index2word_set, num_features)
#### Due to the averaging, there could be infinitive values or NaN values. The next numpy function turns these value to "0" scores
some_chat_embedding_vectors = np.nan_to_num(some_chat_embedding_vectors)  

Review 0 of 9


In [34]:
# have classifier make a prediction
#pred = svm_linear_clf.predict(ourDataVecs)
from sklearn.calibration import CalibratedClassifierCV

some_chat_pred = svm_linear_clf.predict(some_chat_embedding_vectors)
print('System predictions', some_chat_pred)
print('Gold labels', some_chat_labels)
for review, predicted_label in zip(some_chat, some_chat_pred):
    
    print('%s => %s' % (review, 
                        le.classes_[predicted_label]))




System predictions [3 3 0 5 0 4 4 6 3]
Gold labels [3 3 4 5 0 3 0 6 3]
That is sweet of you => joy
You are so funny => joy
Are you a man or a woman? => anger
Chatbots make me sad and feel lonely. => sadness
Your are stupid and boring. => anger
Two thumbs up => neutral
I fell asleep halfway through this conversation => neutral
Wow, I am really amazed. => surprise
You are amazing. => joy


In [35]:
some_chat_pred_probabilities = svm_linear_clf.predict_proba(some_chat_embedding_vectors)
print(some_chat_pred_probabilities)

[[0.02683891 0.00691002 0.00973693 0.44890352 0.4274696  0.03636787
  0.04377315]
 [0.19377594 0.00972677 0.02869718 0.59289415 0.06249003 0.03201222
  0.08040371]
 [0.30233837 0.03863421 0.01427503 0.05800378 0.25326459 0.06331723
  0.27016679]
 [0.11802492 0.03648063 0.027924   0.1195201  0.13414265 0.54618025
  0.01772745]
 [0.4087405  0.15293821 0.00839875 0.02155103 0.11440176 0.19433383
  0.09963592]
 [0.1262005  0.0828798  0.00280905 0.12636135 0.64596725 0.0046714
  0.01111065]
 [0.14235381 0.01937473 0.04323813 0.07555018 0.55819141 0.07492903
  0.08636271]
 [0.04307893 0.02541457 0.02662532 0.23161007 0.13178276 0.04175628
  0.49973206]
 [0.01993648 0.0130976  0.00688361 0.57365992 0.28987442 0.03049902
  0.06604895]]


Using *Pandas* we can nicely visualise the results. 

In [36]:
some_chat_pred_labels = []
for predicted_label in some_chat_pred:
    some_chat_pred_labels.append(le.classes_[predicted_label])

some_chat_gold_labels = []
for gold_label in some_chat_labels:
    some_chat_gold_labels.append(le.classes_[gold_label])


result_frame = pd.DataFrame(some_chat_pred_probabilities*100, columns=le.classes_)
result_frame['Chat']=some_chat
result_frame['Predication']=some_chat_pred_labels
result_frame['Gold']=some_chat_gold_labels

result_frame

,anger,disgust,fear,joy,neutral,sadness,surprise,Chat,Predication,Gold
0,2.683891,0.691002,0.973693,44.890352,42.746960,3.636787,4.377315,That is sweet of you,joy,joy
1,19.377594,0.972677,2.869718,59.289415,6.249003,3.201222,8.040371,You are so funny,joy,joy
2,30.233837,3.863421,1.427503,5.800378,25.326459,6.331723,27.016679,Are you a man or a woman?,anger,neutral
3,11.802492,3.648063,2.792400,11.952010,13.414265,54.618025,1.772745,Chatbots make me sad and feel lonely.,sadness,sadness
4,40.874050,15.293821,0.839875,2.155103,11.440176,19.433383,9.963592,Your are stupid and boring.,anger,anger
5,12.620050,8.287980,0.280905,12.636135,64.596725,0.467140,1.111065,Two thumbs up,neutral,joy
6,14.235381,1.937473,4.323813,7.555018,55.819141,7.492903,8.636271,I fell asleep halfway through this conversation,neutral,anger
7,4.307893,2.541457,2.662532,23.161007,13.178276,4.175628,49.973206,"Wow, I am really amazed.",surprise,surprise
8,1.993648,1.309760,0.688361,57.365992,28.987442,3.049902,6.604895,You are amazing.,joy,joy


In [37]:
report = classification_report(some_chat_labels,some_chat_pred,digits = 7)
print(le.classes_)
print('SVM LINEAR ----------------------------------------------------------------')
print(report)

['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'surprise']
SVM LINEAR ----------------------------------------------------------------
              precision    recall  f1-score   support

           0  0.5000000 0.5000000 0.5000000         2
           3  1.0000000 0.7500000 0.8571429         4
           4  0.0000000 0.0000000 0.0000000         1
           5  1.0000000 1.0000000 1.0000000         1
           6  1.0000000 1.0000000 1.0000000         1

    accuracy                      0.6666667         9
   macro avg  0.7000000 0.6500000 0.6714286         9
weighted avg  0.7777778 0.6666667 0.7142857         9



### 7. Saving the classifier to disk

Just as with the previous notebook, you can save the emotion classification model to disk and load the model some other time. Note that you need to load the same word2vec model as well to represent any text input with vector representations that are compatible.

In [38]:
import pickle

# save the classifier to disk
filename_classifier = './models/svm_linear_clf_embeddings.sav'
pickle.dump(svm_linear_clf, open(filename_classifier, 'wb'))

filename_classifier = './models/svm_nonlinear_clf_embeddings.sav'
pickle.dump(svm_nonlinear_clf, open(filename_classifier, 'wb'))

filename_encoder = './models/label_encoder.sav'
 
# some time later...
 
# load the classifier and the vectorizer from disk
loaded_classifier = pickle.load(open(filename_classifier, 'rb'))
loaded_label_encoder = pickle.load(open(filename_encoder, 'rb'))

# Apply preprocessing
some_chat_tokens = []
for utterance in some_chat:
    some_chat_tokens.append(nltk.tokenize.word_tokenize(utterance))
some_chat_embedding_vectors = getAvgFeatureVecs(some_chat_tokens, stop_words, word_embedding_model, index2word_set, num_features)
some_chat_embedding_vectors = np.nan_to_num(some_chat_embedding_vectors)

# Apply classifier and report predictions
pred_from_loaded_classifier = loaded_classifier.predict(some_chat_embedding_vectors)

# Report predictions
print('System predictions', some_chat_pred)
print('Gold labels', some_chat_labels)
for review, predicted_label in zip(some_chat, some_chat_pred):
    print('%s => %s' % (review, 
                        loaded_label_encoder.classes_[predicted_label]))
    
# Report predictions (probabilities)
some_chat_pred_probabilities = loaded_classifier.predict_proba(some_chat_embedding_vectors)
print(some_chat_pred_probabilities)


Review 0 of 9
System predictions [3 3 0 5 0 4 4 6 3]
Gold labels [3 3 4 5 0 3 0 6 3]
That is sweet of you => joy
You are so funny => joy
Are you a man or a woman? => anger
Chatbots make me sad and feel lonely. => sadness
Your are stupid and boring. => anger
Two thumbs up => neutral
I fell asleep halfway through this conversation => neutral
Wow, I am really amazed. => surprise
You are amazing. => joy
[[0.06242633 0.03354553 0.01709483 0.17764976 0.604794   0.08274854
  0.021741  ]
 [0.09756114 0.01984959 0.03185186 0.17215211 0.58706584 0.03305914
  0.05846033]
 [0.09228869 0.03033759 0.01519324 0.00618809 0.53911847 0.04233373
  0.27454019]
 [0.04894699 0.03526466 0.01985468 0.06069891 0.62965904 0.18741068
  0.01816504]
 [0.1952723  0.05168056 0.01651187 0.00395565 0.58432463 0.08356258
  0.0646924 ]
 [0.09950616 0.03340602 0.0169997  0.17008627 0.60165589 0.04673985
  0.03160612]
 [0.11498147 0.02327063 0.02666716 0.09333747 0.61696948 0.1028548
  0.02191899]
 [0.01741515 0.01760496 

# End of this notebook